<a href="https://colab.research.google.com/github/ADinng/CS6319_MachineLearning/blob/main/CS6319_midterm_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Regression

### 2024midterm_regression

In [ ]:
# 1. import data
import pandas as pd
import numpy as np

url = "https://github.com/andvise/DataAnalyticsDatasets/blob/main/Data.csv?raw=true"
data = pd.read_csv(url)

# 2. check data
data.head()
data.describe()
data.shape

# 3. add column name
data = pd.read_csv(url, header=None)

data[0].value_counts()

# 4. check for miss value
data.isnull().sum()

# 5. hand miss value
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
datanum = data.iloc[:,1:]
datacat = data.iloc[:,0]

imputer.fit(datanum)
X = imputer.transform(datanum)
datanum= pd.DataFrame(X, columns=datanum.columns)
data = pd.concat([datacat, datanum], axis=1)

# 6. split data
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)

# 7. check features (if corr_matrix<0.1 then Remove features that do not correlate with the label)
corr_matrix=trainset.drop(0, axis=1).corr()
print(corr_matrix[8])

# 8. create labs
traindata = trainset.iloc[:,:-1]
trainlabs = trainset.iloc[:,-1]
testdata = testset.iloc[:,:-1]
testlabs = testset.iloc[:,-1]

# 9. create cat and num
traincat = traindata[[0]]
trainnum = traindata.drop(0, axis=1)

testcat = testdata[[0]]
testnum = testdata.drop(0, axis=1)

# 10. use standscaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainnumr = scaler.fit_transform(trainnum)
testnumr = scaler.transform(testnum)

# 11.use one hot encoder
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
trainhot = cat_encoder.fit_transform(traincat).toarray()
trainnew = np.concatenate([trainnumr, trainhot], axis=1)

testhot = cat_encoder.transform(testcat).toarray()
testnew = np.concatenate([testnumr, testhot], axis=1)

# 12.use shuffle and randomforest
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

split = ShuffleSplit(n_splits=10, test_size=0.2)

rfreg = RandomForestRegressor()
scores = cross_val_score(rfreg, trainnew, trainlabs, scoring="neg_mean_squared_error", cv=split)
rmse=np.sqrt(-scores)
print("XVALSHUF RF RMSE mean = ",rmse.mean()," stddev =",rmse.std())

# 12.1 use KNN (rmse越小越好)
from sklearn.neighbors import KNeighborsRegressor
knnreg = KNeighborsRegressor()
scores = cross_val_score(knnreg, trainnew, trainlabs, scoring="neg_mean_squared_error", cv=split)
rmse=np.sqrt(-scores)
print("XVALSHUF KNN RMSE mean = ",rmse.mean()," stddev =",rmse.std())


# 13. use grid search to tune be hyperparameters of the winning regressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

param_grid = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'max_depth': [None, 10, 20, 30],  # 树的最大深度
    'min_samples_split': [2, 5, 10],  # 分裂内部节点所需的最小样本数
    'min_samples_leaf': [1, 2, 4],    # 叶节点所需的最小样本数
    'max_features': ['log2', 'sqrt']   # 寻找最佳分割时考虑的特征数量
}

rfreg = RandomForestRegressor(random_state=42)

# 创建网格搜索对象
grid_search = GridSearchCV(
    estimator=rfreg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # 使用负均方误差作为评分标准
    cv=5,  # 5折交叉验证
    n_jobs=2,  # 使用所有可用的CPU核心
)

grid_search.fit(trainnew, trainlabs)

print("Best parameters found: ", grid_search.best_params_)



# 14. evaluate the best-performing regressor on the test data
best_rfreg = grid_search.best_estimator_

y_pred = best_rfreg.predict(testnew)
test_rmse = np.sqrt(mean_squared_error(testlabs, y_pred))
print("Test RMSE with best model: ", test_rmse)

### lecturenote

In [ ]:
# 1. import data read csv
house = pd.read_csv("https://github.com/ageron/handson-ml/raw/master/datasets/housing/housing.csv")

# 2. examine the dataframe
house.head()
house.info()
house["ocean_proximity"].value_counts() # 检查类别分布
house.describe()

# 3. Split dataset into training and testing data
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(house, test_size=0.2)

# 4. separate the labels (columns (axis 1) or rows (axis 0))
traindata = trainset.drop("median_house_value", axis=1)
trainlabs = trainset["median_house_value"].copy()

# 5.Separate categorical and numerical attributes
traincat = traindata[["ocean_proximity"]]
trainnum = traindata.drop("ocean_proximity", axis=1)

# 6.Handling missing values(处理缺失值) mean均值，median中位数，most_frequent众数
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(trainnum)
X = imputer.transform(trainnum)
trainnum= pd.DataFrame(X, columns=trainnum.columns)

# 7.rescaler to the traindata(数据缩放)StandardScaler (将数据标准化为均值 0 和标准差 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainnumr = scaler.fit_transform(trainnum)

# 8. fit regressor to the training data
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(trainnumr, trainlabs)
predictions=linreg.predict(trainnumr)

# 9. use mse and rmse (rmse 越小，模型预测越准确 ,关注异常值,mae, 稳健易解释的指标)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(trainlabs, predictions)
rmse = np.sqrt(mse)
print("Train numeries LIN error = ", rmse)

# 10. create new features (combining numerical attributes)
trainnum["rooms_per_household"]=trainnum["total_rooms"]/trainnum["households"]
trainnum["bedrooms_per_room"]= trainnum["total_bedrooms"]/trainnum["total_rooms"]
trainnum["population_per_household"]=trainnum["population"]/trainnum["households"]

imputer.fit(trainnum)
X = imputer.transform(trainnum)
trainnum = pd.DataFrame(X, columns=trainnum.columns)
trainnumr = scaler.fit_transform(trainnum)


# 10.1 Feature selection_spearman's corrleation(if < 0.2 can drop, >0.9 or ==1, can drop)
corr_matrix=trainset.drop("ocean_proximity", axis=1).corr()
print(corr_matrix["median_house_value"])

# longitude -0.047998 latitude -0.143619 housing_median_age 0.111647 total_rooms 0.140739 total_bedrooms 0.055268 population -0.018309 households 0.071222 median_income 0.693173 median_house_value 1.000000 Name: median_house_value, dtype: float64 %%

housing.plot(x="median_income", y="median_house_value", kind="scatter", alpha=0.1)

trainnum1 = trainnumr[:, [1, 2, 3, 7, 9, 10]] linreg.fit(trainnum1, trainlabs) predictions = linreg.predict(trainnum1)
mse = mean_squared_error(trainlabs, predictions)
rmse = np.sqrt(mse) print("Train selected numericas LIN error = ", rmse)
# Train selected numericas LIN error = 77316.06904450871 %%


# 11. use onehot encode
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
trainhot = cat_encoder.fit_transform(traincat).toarray()
trainnew = np.concatenate([trainnumr, trainhot], axis=1)

linreg.fit(trainnew, trainlabs)
predictions = linreg.predict(trainnew)
mse = mean_squared_error(trainlabs, predictions)
rmse = np.sqrt(mse)
print("Train combined arrributes LIN error = ", rmse)
# Train combined arrributes LIN error = 67765.74947521035


# 12. other regression
# DecisionTree
from sklearn.tree import DecisionTreeRegressor

treereg = DecisionTreeRegressor()
treereg.fit(trainnew, trainlabs)
predictions = treereg.predict(trainnew)
mse = mean_squared_error(trainlabs, predictions)
rmse = np.sqrt(mse)
print("Train combined attributes DT error = ", rmse)
# Train combined attributes DT error = 0.0

# 13. use testdata for evaluation, and the training data for learning
testdata = testset.drop("median_house_value", axis=1)
testlabs = testset["median_house_value"].copy()

testnum = testdata.drop("ocean_proximity", axis=1)
testcat = testdata[["ocean_proximity"]]

testnum["rooms_per_household"] = testnum["total_rooms"]/testnum["households"]
testnum["bedrooms_per_room"] = testnum["total_bedrooms"]/testnum["total_rooms"]
testnum["population_per_household"] = testnum["population"] / testnum["households"]

X = imputer.transform(testnum)
testnum = pd.DataFrame(X, columns=testnum.columns)
testnumr = scaler.transform(testnum)

testhot = cat_encoder.fit_transform(testcat).toarray()
testnew = np.concatenate([testnumr, testhot], axis=1)

# random forest
from sklearn.ensemble import RandomForestRegressor
rfreg = RandomForestRegressor()
rfreg.fit(trainnew, trainlabs)

predictions = rfreg.predict(testnew)
mse = mean_squared_error(testlabs, predictions)
rmse = np.sqrt(mse)
print("Test combined attributes RF error = ", rmse)
# Test combined attributes RF error = 51474.77108788628

# 14. cross-validation
# 14.1 k-fold cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(linreg, trainnew, trainlabs, scoring="neg_mean_squared_error", cv=10)
rmse = np.sqrt(-scores)
print("XVAL LIN RMSE mean =",rmse.mean()," stddev =",rmse.std())
#XVAL LIN RMSE mean = 67929.13193437677 stddev = 2602.8838783869105

# 14.2 Shuffle split cross-validataion
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(linreg, trainnew, trainlabs, scoring="neg_mean_squared_error", cv=split)
rmse=np.sqrt(-scores)
print("XVALSHUF LIN RMSE mean = ",rmse.mean()," stddev =",rmse.std())


## Classification

### 0.2023midterm classification

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. import data
#(You should take into account the fact that values are separated by spaces instead of commas, and thatthe file has no header.)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data"

data = pd.read_csv(url, sep='\s+', header=None)

# 2. check data
data.head()
data.describe()

# 3.split data
trainset, testset = train_test_split(data, test_size=0.2)

traindata = trainset.iloc[:,1:9]
trainlabs = trainset.iloc[:,9]
testdata = testset.iloc[:,1:9]
testlabs = testset.iloc[:,9]

# 4. check miss value
missing_values = data.isnull().sum()
missing_values

# 5. minmax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
traindatar = scaler.fit_transform(traindata)
testdatar = scaler.transform(testdata)

# 6. LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainlabse = le.fit_transform(trainlabs)
testlabse = le.transform(testlabs)

# 7. LinearSVC
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

classifier = LinearSVC()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, traindatar, trainlabse, scoring="f1_macro", cv=split)
print("LS f1 mean =", scores.mean()," stddev =",scores.std())

# 8.random
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
scores = cross_val_score(classifier, traindatar, trainlabse, scoring="f1_macro", cv = split)
print("RF f1 mean = ", scores.mean()," stddev = ", scores.std())

# 9.KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
scores = cross_val_score(classifier, traindatar, trainlabse, scoring="f1_macro", cv = split)
print("KN f1 mean = ", scores.mean()," stddev = ", scores.std())

# 10.use the best to fit test value
best_clf = KNeighborsClassifier()
best_clf.fit(traindatar, trainlabse)

# Evaluate on the test set
y_pred = best_clf.predict(testdatar)
test_accuracy = accuracy_score(testlabse, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")


LS f1 mean = 0.46000339526240674  stddev = 0.051681193222509504
RF f1 mean =  0.47375143443529816  stddev =  0.053952010212195804
KN f1 mean =  0.5465495971996273  stddev =  0.03785665844100078
Test Accuracy: 0.5926


### 1. binary classification

In [ ]:
# binary classification: electrical grid stability

import pandas as pd
import numpy as np
import sklearn

# 1. import data
griddata = pd.read_csv("https://archive.ics.uci.edu/ml/machinelearning-databases/00471/Data_for_UCI_named.csv")
griddata = griddata.drop("stab", axis=1)
gridvars = griddata.drop("stabf", axis=1)
gridlabs = griddata["stabf"].copy()

# 2.check for missing values
gridvars.info()
gridvars.describe()


# 3.use normalisation or standardisation to make all variables between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
gridvarsr = scaler.fit_transform(gridvars)

# 4. label encode
# sklearn classifiers expect class labels to be in the form 0,1,2,...
# we need to transform the stable/unstable values.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
gridlabse = le.fit_transform(gridlabs)

# 5. fit classifier
# 5.1 linear support vector classifier, instead use accuracy
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

classifier = LinearSVC()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, gridvarsr, gridlabse, scoring="accuracy", cv=split)
print("LS acc mean =",scores.mean()," stddev =",scores.std())


# 5.2 decision tree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, gridvarsr, gridlabse, scoring="accuracy", cv=split)
print("DT acc mean =",scores.mean()," stddev =",scores.std())

# 5.3 random forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, gridvarsr, gridlabse, scoring="accuracy", cv=split)
print("RT acc mean =",scores.mean()," stddev =",scores.std())


# 5.4 Support vector classifier
from sklearn.svm import SVC
classifier = SVC()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, gridvarsr, gridlabse, scoring="accuracy", cv=split)
print("SV acc mean =",scores.mean()," stddev =",scores.std())

# 5.5 K-Nearest-Neighbours
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, gridvarsr, gridlabse, scoring="accuracy", cv=split)
print("KN acc mean =",scores.mean()," stddev =",scores.std())

### Multiple classes

imbalanced data

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
mnisttra = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra",header=None)
mnisttes = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes",header=None)

# split label
mnisttrad = mnisttra.drop(64, axis=1)
mnisttesd = mnisttes.drop(64, axis=1)
mnisttral = mnisttra[64].copy()
mnisttesl = mnisttes[64].copy()
ytra = (mnisttral == 5)
ytes = (mnisttesl == 5)

# label encoder
le = LabelEncoder()
ytrae = le.fit_transform(ytra)

from sklearn.svm import LinearSVC
from sklearn.model_selection import ShuffleSplit, cross_val_score
classifier = LinearSVC()
split = ShuffleSplit(n_splits=10, test_size=0.2)
score = cross_val_score(classifier, mnisttrad, ytrae, scoring="accuracy", cv=split)
print("LS accuracy mena=", score.mean())

# using accuracy isn't appropriate for imbalance data

# decisiontree and f1
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
scores = cross_val_score(classifier, mnisttrad, ytrae, scoring="f1", cv = split)
print("DT f1 mean = ", score.mean()," stddev = ", scores.std())

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
scores = cross_val_score(classifier, mnisttrad, ytrae, scoring="f1", cv = split)
print("RF f1 mean = ", score.mean()," stddev = ", scores.std())


from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
scores = cross_val_score(classifier, mnisttrad, ytrae, scoring="f1", cv = split)
print("KN f1 mean = ", score.mean()," stddev = ", scores.std())


# confusion matrix
mnisttrale = le.fit_transform(mnisttral)
classifier = DecisionTreeClassifier()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, mnisttrad, mnisttrale, scoring="accuracy", cv=split)
print("DT accuracy mean = ",scores.mean(), " stddev = ", scores.std())

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
# generate a confusion matrix for multiclass problem
ypred = cross_val_predict(classifier, mnisttrad, mnisttrale, cv=3)
conf = confusion_matrix(ypred, mnisttrale)
print(conf)

import matplotlib as plt
plt.pyplot.matshow(conf, cmap=plt.cm.gray)
rowsums = conf.sum(axis=1, keepdims=True)
normconf = conf/rowsums
np.fill_diagonal(normconf, 0)

plt.pyplot.matshow(normconf, cmap=plt.cm.gray)


In [ ]:
# 1. import data
firewall = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00542/log2.csv")

#categories very imblanced --> use F1 scoring
firewall["Action"].value_counts()
# 2. split data
from sklearn.model_selection import train_test_split
firetrain, firetest = train_test_split(firewall, test_size=0.2)

# 3. choose labs and drop data
# iloc[row index, colum index] 左闭又开，traindata 只取5-10列
traindata = firetrain.iloc[:,5:11]
trainlabs = firetrain.iloc[:,4]
testdata = firetest.iloc[:,5:11]
testlabs = firetest.iloc[:,4]

# 4. data rescaled (MinMax将值缩放到 [0, 1])
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
traindatar = scaler.fit_transform(traindata)
testdatar = scaler.transform(testdata)

#5. encode the text labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainlabse = le.fit_transform(trainlabs)
testlabse = le.transform(testlabs)

#6. fit data LinearSVC
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

classifier = LinearSVC()
split = ShuffleSplit(n_splits=10, test_size=0.2)
scores = cross_val_score(classifier, traindatar, trainlabse, scoring="f1_macro", cv=split)
print("LS f1 mean =", scores.mean()," stddev =",scores.std())

#7. Grid search
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
params =[{'n_neighbors': [3, 5, 8, 13],
          'weights': ['uniform','distance'],
          'p':[1,2]}]
classifier = KNeighborsClassifier()
search = GridSearchCV(classifier, params, cv=5, scoring ="f1_macro")
search.fit(traindatar, trainlabse)

print("best params: ", search.best_params_)
print("best score: ", search.best_score_)

#best params:  {'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
#best score:  0.7476954686129262

# 7.2 优化
params =[{'n_neighbors': [6, 8, 10, 15, 18]}]
classifier = KNeighborsClassifier(weights='distance',p=1)
search = GridSearchCV(classifier, params, cv=5, scoring ="f1_macro")
search.fit(traindatar, trainlabse)

print("best params: ", search.best_params_)
print("best score: ", search.best_score_)
#best params:  {'n_neighbors': 18}
#best score:  0.7487962725321669

linear classifier

## Regression.gork

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 读取数据
df = pd.read_csv('exam_data.csv')
print(df.head())
df.info()
print(df.describe())

# 拆分特征和标签
X = df.drop('target', axis=1)
y = df['target']

# 拆分训练和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 处理缺失值
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# 数据缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 训练模型
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 交叉验证
scores = cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='neg_mean_squared_error')
print("Cross-val RMSE:", np.sqrt(-scores.mean()))

# 测试集评估
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE:", rmse)

## Classification.gork

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# 读取数据
df = pd.read_csv('exam_data.csv')
print(df.head())
df.info()
print(df['target'].value_counts())  # 检查类别分布

# 拆分特征和标签
X = df.drop('target', axis=1)
y = df['target']

# 拆分训练和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 处理缺失值
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# 标签编码（如果需要）
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# 数据缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 训练模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

# 交叉验证
scores = cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='accuracy')
print("Cross-val Accuracy:", scores.mean())

# 测试集评估
y_pred = model.predict(X_test_scaled)
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Macro Score:", f1)